# Roteamento

In [1]:
import os

from dotenv import find_dotenv, load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
# * Carrega as variáveis de ambiente
_ = load_dotenv(find_dotenv())

# * Verifica se a API key está configurada
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# * Configura o modelo com parâmetros específicos
model: ChatOpenAI = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.0,  # Controla a criatividade das respostas
)  # type: ignore

response = model.invoke("The Sky is ?")
print(response.content)

The sky is **blue** (during the day) due to the scattering of sunlight by the atmosphere. This phenomenon is called **Rayleigh scattering**. At sunrise and sunset, the sky can appear **red, orange, or pink** because the sunlight passes through more of the atmosphere, scattering shorter blue wavelengths and allowing longer red wavelengths to dominate.

At night, the sky appears **dark or black**, dotted with stars, because the sun is no longer illuminating the atmosphere from your location.

So, the sky **can be blue, red, orange, pink, gray, or black** depending on the time of day, weather, and atmospheric conditions.


In [3]:
prompt = ChatPromptTemplate.from_template("""Você é um professor de matemática de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_matematica = prompt | model

prompt = ChatPromptTemplate.from_template("""Você é um professor de física de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_fisica = prompt | model

prompt = ChatPromptTemplate.from_template("""Você é um professor de história de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_historia = prompt | model

prompt = ChatPromptTemplate.from_template("""{pergunta}""")
chain_generica = prompt | model

In [4]:
from pydantic import BaseModel, Field

prompt = ChatPromptTemplate.from_template(
    "Você deve categorizar a seguinte pergunta: {pergunta}"
)


class Categorizador(BaseModel):
    """Categoriza as perguntas de alunos do ensino fundamental"""

    area_conhecimento: str = Field(
        description='A área de conhecimento da pergunta feita pelo aluno. \
    Deve ser "física", "matemática" ou "história". Caso não se encaixe em nenhuma delas, retorne "outra"'
    )


model_estruturado = prompt | model.with_structured_output(Categorizador)
model_estruturado.invoke({"pergunta": "Quando foi a inependencia dos estados unidos?"})

Categorizador(area_conhecimento='história')

### Criando estrutura de roteamento

In [5]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(categoria=model_estruturado)
chain.invoke({"pergunta": "Quando foi a independencia dos estados unidos?"})

{'pergunta': 'Quando foi a independencia dos estados unidos?',
 'categoria': Categorizador(area_conhecimento='história')}

In [ ]:
def route(input):
    if input["categoria"].area_conhecimento == "matemática":
        return chain_matematica
    if input["categoria"].area_conhecimento == "física":
        return chain_fisica
    if input["categoria"].area_conhecimento == "história":
        return chain_historia
    return chain_generica

In [ ]:
chain = RunnablePassthrough().assign(categoria=model_estruturado) | route
chain.invoke({"pergunta": "Quando foi a inependencia dos estados unidos?"})

In [ ]:
chain.invoke({"pergunta": "Quanto é 1 + 21?"})